In [8]:

import os
import torch
import pandas as pd
import numpy as np
import random
import torch.nn as nn
import matplotlib.pyplot as plt
from glob import glob
import time
from tqdm import tqdm
from sklearn import metrics
import cv2
import gc
import torch.nn.functional as F
from glob import glob
from sklearn.model_selection import KFold
from datetime import datetime
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from sklearn.datasets.samples_generator import make_blobs

In [3]:
SEED = 42
def seed_everything(seed):
    print(f'setting everything to seed {seed}')
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

setting everything to seed 42


# EDA

In [4]:
train_data = np.load('data_hw2/train_data.npy')
train_label = np.load('data_hw2/train_label.npy')
test_data = np.load('data_hw2/test_data.npy')
test_label = np.load('data_hw2/test_label.npy')

In [5]:
features = [f'f_{i}' for i in range(test_data.shape[1])]
test_df = pd.DataFrame(test_data, columns=features)
test_df['label'] = test_label
test_df.label.value_counts()

 1.0    4692
-1.0    4480
 0.0    4416
Name: label, dtype: int64

In [6]:
features = [f'f_{i}' for i in range(train_data.shape[1])]
train_df = pd.DataFrame(train_data, columns=features)
train_df['label'] = train_label
train_df.label.value_counts()

 1.0    12903
-1.0    12320
 0.0    12144
Name: label, dtype: int64

In [9]:
train_df.describe()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_301,f_302,f_303,f_304,f_305,f_306,f_307,f_308,f_309,label
count,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,...,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000,37367.000000
mean,26.307912,22.110035,20.304931,18.683185,17.780938,26.308013,22.096060,20.280133,18.423180,17.359387,...,20.958111,19.752555,18.257986,16.787448,25.440076,21.212016,19.943632,18.456378,16.906975,0.015602
std,0.921825,0.815095,0.790141,0.889276,1.289960,0.984163,0.818914,0.784670,0.634411,0.895656,...,0.605819,0.739794,0.553449,0.911842,0.936969,0.528262,0.646587,0.467847,0.829632,0.821451
min,23.986169,20.090675,18.458207,17.111253,15.401767,23.757498,20.126303,18.480260,16.973346,15.099613,...,19.469145,18.385926,17.079901,15.089669,23.538066,20.068334,18.770023,17.480364,15.413249,-1.000000
25%,25.718730,21.534643,19.738649,18.106129,16.924630,25.653640,21.545284,19.755248,18.037299,16.785763,...,20.476930,19.191609,17.871323,16.141387,24.824087,20.753884,19.455725,18.126238,16.288857,-1.000000
50%,26.295942,22.173384,20.296463,18.439192,17.360649,26.384671,22.188781,20.341341,18.412210,17.248337,...,20.948789,19.651535,18.190214,16.604118,25.418905,21.170567,19.838847,18.391835,16.802563,0.000000
75%,26.938378,22.682997,20.820196,18.944697,18.260674,27.000940,22.658471,20.810279,18.795881,17.873497,...,21.460246,20.119153,18.719843,17.405790,25.920063,21.680352,20.270002,18.815960,17.381670,1.000000
max,31.117939,24.459037,22.433177,21.432797,21.636593,31.058733,24.359110,22.205323,20.029018,19.701286,...,22.184486,22.255170,20.315344,20.470531,30.768231,22.318322,22.212960,20.281005,19.980844,1.000000


In [21]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(train_df.sample(2000)[features])
pcas = pca.transform(train_df.sample(10000)[features])

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
fig = plt.figure()
ax = Axes3D(fig, elev=30, azim=20)
plt.scatter(pcas[:, 0], pcas[:, 1], pcas[:, 2],marker='o')

In [48]:
# 3397个样本对应一个人。 训练集11人
for i in range(11):
    train_df.loc[range(i*3397,(i+1)*3397),'people'] = i

In [67]:
from sklearn.model_selection import StratifiedKFold,GroupKFold
kfold = GroupKFold(n_splits=5)
train_df['fold'] = 0
for fold,(train_idx,val_idx) in enumerate(kfold.split(train_df,train_df['label'],groups=train_df.people)):
    train_df.loc[val_idx,'fold'] = fold
train_df.groupby('fold').size()

fold
0    10191
1     6794
2     6794
3     6794
4     6794
dtype: int64

In [61]:
# from sklearn.model_selection import StratifiedKFold,GroupKFold
# kfold = StratifiedKFold(n_splits=5)
# train_df['fold'] = 0
# for fold,(train_idx,val_idx) in enumerate(kfold.split(train_df,train_df['label'])):
#     train_df.loc[val_idx,'fold'] = fold
# train_df.groupby('fold').size()

fold
0    7474
1    7474
2    7474
3    7473
4    7472
dtype: int64

In [76]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
# 训练
def train_process(train_df,fold,label='label',istest=True):
    svc=SVC(kernel='rbf',C=100,verbose=True)
    train = train_df[train_df.fold!=0]
    val   = train_df[train_df.fold==0]
    train = shuffle(train)
    traindata = train[features].values
    trainlabel = train[label].values
    valdata = val[features].values
    vallabel = val[label].values
    testdata = test_df[features].values
    testlabel = test_df[label].values
    # 标准化
    ss = StandardScaler()
    ss.fit(traindata)
    X_train_std = ss.transform(traindata)
    ss.fit(valdata)
    X_val_std = ss.transform(valdata)
    ss.fit(testdata)
    X_test_std = ss.transform(testdata)
    # 训练
    svc.fit(X_train_std, trainlabel)
    # 验证
    y_pred=svc.predict(X_val_std)
    print('val acc:',(accuracy_score(vallabel, y_pred)))
    # 测试
    y_pred=svc.predict(X_test_std)
    print('test acc:',accuracy_score(testlabel, y_pred))
    return y_pred

In [71]:
train_process(train_df,0)

[LibSVM] 0.4886664704150721
test acc: 0.5679275831616132


In [72]:
# 分割 one vs rest
# -1->[0,1]
train_df['lb1'] = train_df.label.apply(lambda x: 1 if x==-1 else 0)
train_df['lb2'] = train_df.label.apply(lambda x: 1 if x==0 else 0)
train_df['lb3'] = train_df.label.apply(lambda x: 1 if x==1 else 0)

In [73]:
test_df['lb1'] = test_df.label.apply(lambda x: 1 if x==-1 else 0)
test_df['lb2'] = test_df.label.apply(lambda x: 1 if x==0 else 0)
test_df['lb3'] = test_df.label.apply(lambda x: 1 if x==1 else 0)

In [78]:
pred_lb1 = train_process(train_df,0,label='lb1')
pred_lb2 = train_process(train_df,0,label='lb2')
pred_lb3 = train_process(train_df,0,label='lb3')

[LibSVM]val acc: 0.586792267687175
test acc: 0.6496173093906388
[LibSVM]val acc: 0.5774703169463252
test acc: 0.745216367382985
[LibSVM]val acc: 0.8066921793739574
test acc: 0.7593464821901678


In [88]:
finalpred = np.stack([pred_lb1,pred_lb2,pred_lb3])
finalpred = finalpred.argmax(axis=0)
finalpred = finalpred-1
print('test acc:',accuracy_score(testlabel, finalpred))

In [101]:
# MIN-MAX  part-part
# 先分成one vs rest 再用随机分解和先验知识分解 分为平衡的二分类问题

In [141]:
def train_process(train_df,fold,label='label',istest=True):
#     label = 'lb1'
    # rest
    # split = 'lb2'
    svc=SVC(kernel='rbf',C=100,verbose=True)
    train = train_df[train_df.fold!=0]
    val   = train_df[train_df.fold==0]
    train = shuffle(train)

    # 随机分成平衡的两部分训练

    # 第一部分
    train_p =  train[train[label]==1].sample(8000)
    train_s =  train[train[label]==0].sample(12000)
    train_1 = train_p.append(train_s)
    traindata = train_1[features].values
    trainlabel = train_1[label].values

    valdata = val[features].values
    vallabel = val[label].values
    testdata = test_df[features].values
    testlabel = test_df[label].values
    # 标准化
    ss = StandardScaler()
    ss.fit(traindata)
    X_train_std = ss.transform(traindata)
    ss.fit(valdata)
    X_val_std = ss.transform(valdata)
    ss.fit(testdata)
    X_test_std = ss.transform(testdata)
    # 训练
    svc.fit(X_train_std, trainlabel)
    # 验证
    y_pred=svc.predict(X_val_std)
    print('val acc:',(accuracy_score(vallabel, y_pred)))
    # 测试
    y_pred1=svc.predict(X_test_std)
    print('test acc:',accuracy_score(testlabel, y_pred1))

    # 第二部分
    train_p =  train[train[label]==1].sample(8000)
    train_s =  train[train[label]==0].sample(12000)
    train_2 = train_p.append(train_s)
    traindata = train_2[features].values
    trainlabel = train_2[label].values
    svc=SVC(kernel='rbf',C=100,verbose=True)
    # 标准化
    ss = StandardScaler()
    ss.fit(traindata)
    X_train_std = ss.transform(traindata)
    ss.fit(valdata)
    X_val_std = ss.transform(valdata)
    ss.fit(testdata)
    X_test_std = ss.transform(testdata)
    # 训练
    svc.fit(X_train_std, trainlabel)
    # 验证
    y_pred=svc.predict(X_val_std)
    print('val acc:',(accuracy_score(vallabel, y_pred)))
    # 测试
    y_pred2=svc.predict(X_test_std)
    print('test acc:',accuracy_score(testlabel, y_pred2))

    return y_pred1,y_pred2

In [145]:
y1,y2 = train_process(train_df,0,label='lb1')
final1 = (y1 & y2)
y1,y2 = train_process(train_df,0,label='lb2')
final2 = (y1 & y2)
y1,y2 = train_process(train_df,0,label='lb3')
final3 = (y1 & y2)

[LibSVM]val acc: 0.5917966833480522
test acc: 0.6500588754783633
[LibSVM]val acc: 0.5931704445098617
test acc: 0.6502060641742714
[LibSVM]val acc: 0.5660877244627612
test acc: 0.7411686782455107
[LibSVM]val acc: 0.5648120890982239
test acc: 0.7421254047689138
[LibSVM]val acc: 0.814934746344814
test acc: 0.770974389166912
[LibSVM]val acc: 0.8147384947502698
test acc: 0.7714895496025905


In [153]:
finalpred = np.stack([final1,final2,final3])
finalpred = finalpred.argmax(axis=0)
finalpred = finalpred-1
print('test acc:',accuracy_score(test_df.label, finalpred))

test acc: 0.5205328230791875


In [154]:
# 第二种分配方式
def train_process(train_df,fold,label='label',istest=True):
#     label = 'lb1'
    # rest
    lbs = ['lb1','lb2','lb3']
    lbs.remove(label)
    svc=SVC(kernel='rbf',C=100,verbose=True)
    train = train_df[train_df.fold!=0]
    val   = train_df[train_df.fold==0]
    train = shuffle(train)

    # 先验知识分成平衡的两部分训练
    # 第一部分
    split = lbs[0]
    train_p =  train[train[label]==1]
    train_s =  train[(train[label]==0) & (train[split]==1)]
    train_1 = train_p.append(train_s)
    traindata = train_1[features].values
    trainlabel = train_1[label].values

    valdata = val[features].values
    vallabel = val[label].values
    testdata = test_df[features].values
    testlabel = test_df[label].values
    # 标准化
    ss = StandardScaler()
    ss.fit(traindata)
    X_train_std = ss.transform(traindata)
    ss.fit(valdata)
    X_val_std = ss.transform(valdata)
    ss.fit(testdata)
    X_test_std = ss.transform(testdata)
    # 训练
    svc.fit(X_train_std, trainlabel)
    # 验证
    y_pred=svc.predict(X_val_std)
    print('val acc:',(accuracy_score(vallabel, y_pred)))
    # 测试
    y_pred1=svc.predict(X_test_std)
    print('test acc:',accuracy_score(testlabel, y_pred1))

    # 第二部分
    split = lbs[1]
    train_p =  train[train[label]==1]
    train_s =  train[(train[label]==0) & (train[split]==1)]
    train_2 = train_p.append(train_s)
    traindata = train_2[features].values
    trainlabel = train_2[label].values
    svc=SVC(kernel='rbf',C=100,verbose=True)
    # 标准化
    ss = StandardScaler()
    ss.fit(traindata)
    X_train_std = ss.transform(traindata)
    ss.fit(valdata)
    X_val_std = ss.transform(valdata)
    ss.fit(testdata)
    X_test_std = ss.transform(testdata)
    # 训练
    svc.fit(X_train_std, trainlabel)
    # 验证
    y_pred=svc.predict(X_val_std)
    print('val acc:',(accuracy_score(vallabel, y_pred)))
    # 测试
    y_pred2=svc.predict(X_test_std)
    print('test acc:',accuracy_score(testlabel, y_pred2))

    return y_pred1,y_pred2

In [155]:
y1,y2 = train_process(train_df,0,label='lb1')
final1 = (y1 & y2)
y1,y2 = train_process(train_df,0,label='lb2')
final2 = (y1 & y2)
y1,y2 = train_process(train_df,0,label='lb3')
final3 = (y1 & y2)

[LibSVM]val acc: 0.4397016975762928
test acc: 0.4698263173388284
[LibSVM]val acc: 0.5746246688254342
test acc: 0.5183985869885193
[LibSVM]val acc: 0.5548032577764694
test acc: 0.7144539299381808
[LibSVM]val acc: 0.6580315965067216
test acc: 0.6456432146011186
[LibSVM]val acc: 0.7641055833578648
test acc: 0.8085811009714454
[LibSVM]val acc: 0.7399666372289275
test acc: 0.8042390344421548


In [156]:
finalpred = np.stack([final1,final2,final3])
finalpred = finalpred.argmax(axis=0)
finalpred = finalpred-1
print('test acc:',accuracy_score(test_df.label, finalpred))

test acc: 0.5901530762437445
